In [1]:
import requests
import pandas as pd
import pprint
from datetime import datetime, timedelta, timezone

# Define your Prometheus query
prometheus_url = "https://<thanosuser>:<thanospass>@<thanoshost>/api/v1/query_range"
start = datetime.now(timezone.utc) - timedelta(days=21)
end = datetime.now(timezone.utc)

query = 'sum by (service, method, path, code) (agg:service_gin_requests_count:rate_5m{path="/v5/order/authorization/capture", code="200"})'
pprint.pp(start.timestamp())
pprint.pp(end.timestamp())

# Fetch the data
params = {
    'query': query,
    'start': start.isoformat(),
    'end': end.isoformat(),
    'step': '30m',
}

response = requests.get(prometheus_url, params=params)

#pprint.pp(response.content)
data = response.json()['data']['result']

# Process the data into a DataFrame
timestamps = []
values = []

for result in data:
    for value in result['values']:
        timestamps.append(pd.to_datetime(value[0], unit='s'))
        values.append(float(value[1]))

df = pd.DataFrame({'ds': timestamps, 'y': values})

pprint.pp(df)

1725467178.609081
1727281578.609101
                      ds         y
0    2024-09-04 16:00:00  0.140433
1    2024-09-04 16:30:00  0.114815
2    2024-09-04 17:00:00  0.133333
3    2024-09-04 17:30:00  0.148758
4    2024-09-04 18:00:00  0.088662
...                  ...       ...
1004 2024-09-25 14:00:00  0.049507
1005 2024-09-25 14:30:00  0.099999
1006 2024-09-25 15:00:00  0.070198
1007 2024-09-25 15:30:00  0.254989
1008 2024-09-25 16:00:00  0.158533

[1009 rows x 2 columns]


In [13]:
from prophet import Prophet

model = Prophet(daily_seasonality=True, weekly_seasonality=True, yearly_seasonality=True)
model.fit(df)
future = model.make_future_dataframe(
    periods=900,
    freq="1min",
    include_history=False
)
forecasted = model.predict(future)
# forecasted["timestamp"] = forecasted["ds"].map(lambda x: x.timestamp())
# forecasted = forecasted[["timestamp", "yhat", "yhat_lower", "yhat_upper"]]
# forecasted = forecasted.set_index("timestamp")
# pprint.pp(forecasted)
fig = future.plot(forecasted)

12:33:37 - cmdstanpy - INFO - Chain [1] start processing
12:33:37 - cmdstanpy - INFO - Chain [1] done processing


KeyError: "['timestamp'] not in index"